In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
folder="/content/drive/MyDrive/資料科學自學聖經/Fan_AOI_Austin"

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set up paths
base_dir = "/content/drive/MyDrive/資料科學自學聖經/Fan_AOI_Austin/Photos"

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 20% for validation

# Load training data
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',  # 'binary' because it's a 2-class problem (GO/NG)
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])



# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


# Save the trained model
model.save("fan_model.h5")

model.summary()


Found 270 images belonging to 2 classes.
Found 66 images belonging to 2 classes.
Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 638ms/step - accuracy: 0.5640 - loss: 0.7823 - val_accuracy: 0.6212 - val_loss: 0.6602
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 349ms/step - accuracy: 0.6666 - loss: 0.5439 - val_accuracy: 0.8030 - val_loss: 0.6271
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 567ms/step - accuracy: 0.8659 - loss: 0.3705 - val_accuracy: 0.8030 - val_loss: 0.5158
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 352ms/step - accuracy: 0.9630 - loss: 0.2089 - val_accuracy: 0.9091 - val_loss: 0.4621
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 495ms/step - accuracy: 0.9845 - loss: 0.0937 - val_accuracy: 0.4697 - val_loss: 0.6827
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 354ms/step - accuracy: 0.9798 - loss: 0.0569 - val_accuracy: 0.8636 - val_loss: 0.2759
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 336ms/step - accuracy: 0.9849 - loss: 0.0631 - val_accuracy: 0.8788 - val_loss: 0.3816
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 78, 78, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 39, 39, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 37, 37, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 18, 18, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 20736)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       2,654,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,021,573 (30.60 MB)

 Trainable params: 2,673,857 (10.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,347,716 (20.40 MB)

In [6]:
import cv2
import numpy as np
import tensorflow as tf
import glob
import os

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    prediction = model.predict(img)

    # Handle cases where the prediction might not be well-formed-------test
    if prediction is not None and len(prediction) > 0:
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold for binary classification
        confidence = prediction[0][0] * 100  # Confidence score in percentage
        return class_idx, confidence
    else:
        return None, None

# Function to classify all images in given directories
def classify_images_in_directories(directories):
    for directory_path in directories:
        print(f"\nClassifying images in directory: {directory_path}")

        # Use glob to find all image files (supports .jpg, .png, .jpeg)
        image_files = glob.glob(os.path.join(directory_path, "*.jpg")) + \
                      glob.glob(os.path.join(directory_path, "*.png")) + \
                      glob.glob(os.path.join(directory_path, "*.jpeg"))

        if not image_files:
            print("No images found in the directory!")
            continue

        for image_path in image_files:
            result, confidence = classify_image(image_path)
            if result is not None:
                if result == 1:
                    print(f"NG (Defective) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
                    # print(f"GO (Correctly Aligned) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
                else:
                    print(f"GO (Correctly Aligned) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
                    # print(f"NG (Defective) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
            else:
                print(f"Unable to classify image: {os.path.basename(image_path)}")

# Example usage: Classify images from both "60" and "100" folders

directories = "/content/drive/MyDrive/資料科學自學聖經/Fan_AOI_Austin/Photos/60", "/content/drive/MyDrive/資料科學自學聖經/Fan_AOI_Austin/Photos/100"
# base_dir = "C:/OneDrive/! Juxer/TVDI/趙雲瀚老師/Workplace/Project/Fan_AOI_Austin/Photos"
classify_images_in_directories(directories)



Classifying images in directory: /content/drive/MyDrive/資料科學自學聖經/Fan_AOI_Austin/Photos/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
NG (Defective) with 73.95% confidence for image: image039.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
NG (Defective) with 99.83% confidence for image: image050.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
NG (Defective) with 99.82% confidence for image: image062.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
NG (Defective) with 96.24% confidence for image: image043.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
NG (Defective) with 99.91% confidence for image: image057.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
NG (Defective) with 99.79% confidence for image: image058.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
NG (Defective) with 99.83% confidence for image: image055.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
NG (Defective) with 99.82% confidence for image: image046.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
NG (Defective) with 99.93% confidence for image: image063.png
1/1 ━━━━

In [7]:
!pip install gradio # Install the gradio library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


# The code needs to import classify function

In [8]:
import cv2
import numpy as np
import tensorflow as tf
import gradio as gr
from PIL import Image

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image):
    # Convert the image to a format suitable for OpenCV
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize

    # Make prediction
    prediction = model.predict(img)

    # Classification logic
    if prediction is not None and len(prediction) > 0:
        # Set threshold for GO/NG classification
        # Adjusted based on expected results for the provided images
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold set to 0.6
        # confidence = prediction[0][0] * 100  # Confidence score in percentage

        if class_idx == 1:
            result_text = f"<div style='color: red; font-size: 24px; font-weight: bold;'>NG</div>"
            return result_text
        else:
            result_text = f"<div style='color: green; font-size: 24px; font-weight: bold;'>GO</div>"
            return result_text

    else:
        return "<div style='color: black; font-size: 20px;'>Unable to classify image</div>"

# Create Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.HTML(label="Classification Result"),
    title="Cooling Fan GO/NG Inspection",
    description="Upload an image of a cooling fan to check if it is correctly aligned (GO) or defective (NG).",
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a7b1dd559ed28c8dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
